In [4]:
import os
from utils import *
import optuna

In [5]:
data, usermap, itemmap, users = load_data2()
data_train, data_test, data_val=split_data2(data, 0.2, 0.2)

In [6]:
study_name = "rating+ranking"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2024-01-05 11:46:27,402] A new study created in memory with name: rating+ranking


In [7]:
from Evaluation.Evaluator import EvaluatorHoldout

cutoff_list=[5, 10, 15]

evaluator_validation = EvaluatorHoldout(data_val, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(data_test, cutoff_list=cutoff_list)

EvaluatorHoldout: Ignoring 2182 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2631 (20.8%) Users that have less than 1 test interactions


In [10]:
from Recommenders.MatrixFactorization.Cython.MatrixFactorizationImpressions_Cython import MatrixFactorization_FunkSVD_Cython

funk_svd_recommender = MatrixFactorization_FunkSVD_Cython(data_train)
funk_svd_recommender.fit(epochs=200)

MatrixFactorization_FunkSVD_Cython_Recommender: URM Detected 474 ( 3.8%) users with no interactions.
MatrixFactorization_FunkSVD_Cython_Recommender: URM Detected 325 ( 1.5%) items with no interactions.


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations